In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
import random
from itertools import product
import torch

In [23]:
from sklearn.metrics import mean_absolute_error


In [3]:
!pip install neuralforecast wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 285.8/285.8 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 825.4/825.4 kB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 87.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 64.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
import wandb
import numpy as np
import pandas as pd
from neuralforecast.models import DLinear
from neuralforecast.core import NeuralForecast
from neuralforecast.utils import AirPassengersDF
from neuralforecast.losses.pytorch import MAE

# **Read Data**

In [5]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# Train Data:
# Store - the store number
# Dept - the department number
# Date - the week
# Weekly_Sales -  sales for the given department in the given store
# IsHoliday - whether the week is a special holiday week
import pandas as pd

train_df = pd.read_csv('/content/drive/MyDrive/Walmart_Recruiting/Data/train.csv.zip')
train_df.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday
0,1,1,2010-02-05,24924.50,False
1,1,1,2010-02-12,46039.49,True
2,1,1,2010-02-19,41595.55,False
3,1,1,2010-02-26,19403.54,False
4,1,1,2010-03-05,21827.90,False


In [7]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 421570 entries, 0 to 421569
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Store         421570 non-null  int64  
 1   Dept          421570 non-null  int64  
 2   Date          421570 non-null  object 
 3   Weekly_Sales  421570 non-null  float64
 4   IsHoliday     421570 non-null  bool   
dtypes: bool(1), float64(1), int64(2), object(1)
memory usage: 13.3+ MB


In [8]:
# Test Data:
# Store - the store number
# Dept - the department number
# Date - the week
# IsHoliday - whether the week is a special holiday week

test_df = pd.read_csv('/content/drive/MyDrive/Walmart_Recruiting/Data/test.csv.zip')
test_df.head()

,Store,Dept,Date,IsHoliday
0,1,1,2012-11-02,False
1,1,1,2012-11-09,False
2,1,1,2012-11-16,False
3,1,1,2012-11-23,True
4,1,1,2012-11-30,False


In [9]:
# Info about stores
stores_df = pd.read_csv('/content/drive/MyDrive/Walmart_Recruiting/Data/stores.csv')
stores_df.head()

,Store,Type,Size
0,1,A,151315
1,2,A,202307
2,3,B,37392
3,4,A,205863
4,5,B,34875


In [10]:
stores_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45 entries, 0 to 44
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Store   45 non-null     int64 
 1   Type    45 non-null     object
 2   Size    45 non-null     int64 
dtypes: int64(2), object(1)
memory usage: 1.2+ KB


In [11]:
# Features data:
# Store - the store number
# Date - the week
# Temperature - average temperature in the region
# Fuel_Price - cost of fuel in the region
# MarkDown1-5 - anonymized data related to promotional markdowns that Walmart is running. MarkDown data is only available after Nov 2011, and is not available for all stores all the time. Any missing value is marked with an NA.
# CPI - the consumer price index
# Unemployment - the unemployment rate
# IsHoliday - whether the week is a special holiday week

features_df = pd.read_csv('/content/drive/MyDrive/Walmart_Recruiting/Data/features.csv.zip')
features_df.head()

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday
0,1,2010-02-05,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False
1,1,2010-02-12,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,True
2,1,2010-02-19,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,False
3,1,2010-02-26,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,False
4,1,2010-03-05,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,False


In [12]:
features_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8190 entries, 0 to 8189
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Store         8190 non-null   int64  
 1   Date          8190 non-null   object 
 2   Temperature   8190 non-null   float64
 3   Fuel_Price    8190 non-null   float64
 4   MarkDown1     4032 non-null   float64
 5   MarkDown2     2921 non-null   float64
 6   MarkDown3     3613 non-null   float64
 7   MarkDown4     3464 non-null   float64
 8   MarkDown5     4050 non-null   float64
 9   CPI           7605 non-null   float64
 10  Unemployment  7605 non-null   float64
 11  IsHoliday     8190 non-null   bool   
dtypes: bool(1), float64(9), int64(1), object(1)
memory usage: 712.0+ KB


# **Processing**

In [13]:
df = train_df.copy()  # Use consistent variable name

# Add datetime and unique series ID
df['ds'] = pd.to_datetime(df['Date'])  # Convert Date to datetime
df['unique_id'] = df['Store'].astype(str) + "_" + df['Dept'].astype(str)  # unique time series
df['y'] = df['Weekly_Sales']  # Forecast target

# Keep only required columns and sort
df = df[['unique_id', 'ds', 'y']].sort_values(['unique_id', 'ds']).reset_index(drop=True)


In [14]:
def train_val_split(df, test_weeks=39):
    """Split data into train/val sets chronologically"""
    train_dfs = []
    val_dfs = []

    for uid in df['unique_id'].unique():
        series_df = df[df['unique_id'] == uid].sort_values('ds')

        # Take last test_weeks for validation
        val_series = series_df.tail(test_weeks)
        train_series = series_df.head(len(series_df) - test_weeks)

        train_dfs.append(train_series)
        val_dfs.append(val_series)

    return pd.concat(train_dfs).reset_index(drop=True), pd.concat(val_dfs).reset_index(drop=True)

train_df_split, val_df = train_val_split(df)

# **Train DLinear Model**

In [15]:
wandb.init(
    project="walmart-dlinear-forecast",
    name="dlinear_run_02",
    config={
        "input_size": 80,
        "horizon": 39,
        "learning_rate": 1e-3,
        "max_steps": 1000,
        "loss": "mae"
    }
)
config = wandb.config

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: tekla-tsutskha (tekla-tsutskha-free-university-of-tbilisi-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [16]:
def prepare_data(df):
    """Ensure proper data types and handle missing values"""
    df = df.copy()
    df['ds'] = pd.to_datetime(df['ds'])
    df['y'] = pd.to_numeric(df['y'], errors='coerce')
    df['unique_id'] = df['unique_id'].astype(str)

    # Remove any rows with NaN values
    df = df.dropna()

    return df[['unique_id', 'ds', 'y']]

train_df_clean = prepare_data(train_df_split)
val_df_clean = prepare_data(val_df)

print(f"Training data shape: {train_df_clean.shape}")
print(f"Validation data shape: {val_df_clean.shape}")
print(f"Number of unique series: {train_df_clean['unique_id'].nunique()}")

Training data shape: (301297, 3)
Validation data shape: (121455, 3)
Number of unique series: 3086


In [17]:
model = DLinear(
    input_size=config.input_size,
    h=config.horizon,
    learning_rate=config.learning_rate,
    loss=MAE(),
    max_steps=config.max_steps
)


INFO:lightning_fabric.utilities.seed:Seed set to 1


In [18]:
# Initialize NeuralForecast with correct frequency
nf = NeuralForecast(models=[model], freq='W')

# Fit the model
print("Training model...")
nf.fit(df=train_df_clean)

Training model...


INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.callbacks.model_summary:
  | Name          | Type          | Params | Mode 
--------------------------------------------------------
0 | loss          | MAE           | 0      | train
1 | padder_train  | ConstantPad1d | 0      | train
2 | scaler        | TemporalNorm  | 0      | train
3 | decomp        | SeriesDecomp  | 0      | train
4 | linear_trend  | Linear        | 3.2 K  | train
5 | linear_season | Linear        | 3.2 K  | train
--------------------------------------------------------
6.3 K     Trainable params
0         Non-trainable params
6.3 K     Total params
0.025     Total estimated model params size (MB)
8         Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_steps=1000` reached.


In [19]:
print("Generating predictions...")
preds = nf.predict()

Generating predictions...


INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

In [20]:
def evaluate_predictions(preds, val_df, model_name='DLinear'):
    """Evaluate predictions against validation data"""

    # The model name in predictions might be 'DLinear' not 'TFT'
    pred_col = model_name
    if pred_col not in preds.columns:
        # Find the prediction column (usually the model name)
        pred_cols = [col for col in preds.columns if col not in ['unique_id', 'ds']]
        if pred_cols:
            pred_col = pred_cols[0]
        else:
            raise ValueError("No prediction column found in predictions")

    # Merge predictions with validation data
    merged = preds.merge(val_df, on=['unique_id', 'ds'], how='inner')

    if merged.empty:
        raise ValueError("No matching records found between predictions and validation data")

    # Extract true and predicted values
    y_true = merged['y'].values
    y_pred = merged[pred_col].values
    # Calculate metrics
    mae = mean_absolute_error(y_true, y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100

    return mae, mape, merged

In [21]:
try:
    mae, mape, evaluation_df = evaluate_predictions(preds, val_df_clean)

    print(f"\nValidation Results:")
    print(f"MAE: {mae:.4f}")
    print(f"MAPE: {mape:.2f}%")

    # Log metrics to W&B
    wandb.log({
        "val_mae": mae,
        "val_mape": mape
    })

except Exception as e:
    print(f"Evaluation error: {e}")
    print("Prediction columns:", preds.columns.tolist())
    print("Prediction shape:", preds.shape)
    print("Validation shape:", val_df_clean.shape)

Evaluation error: No matching records found between predictions and validation data
Prediction columns: ['unique_id', 'ds', 'DLinear']
Prediction shape: (120354, 3)
Validation shape: (121455, 3)


In [27]:
def analyze_results(evaluation_df, pred_col='DLinear'):
    """Analyze prediction results by series"""
    if evaluation_df.empty:
        return

    # Calculate MAE by series
    series_mae = evaluation_df.groupby('unique_id').apply(
        lambda x: mean_absolute_error(x['y'], x[pred_col])
    ).sort_values(ascending=False)

    print(f"\nTop 5 series with highest MAE:")
    print(series_mae.head())

    print(f"\nTop 5 series with lowest MAE:")
    print(series_mae.tail())

    return series_mae

# Run analysis if evaluation was successful
if 'evaluation_df' in locals():
    pred_column = [col for col in preds.columns if col not in ['unique_id', 'ds']][0]
    series_performance = analyze_results(evaluation_df, pred_column)

wandb.finish()